## prework

* import basic dependencies
* load data
* check data

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pm25prediction/predict.csv
/kaggle/input/pm25prediction/train.csv
/kaggle/input/pm25prediction/test.csv


In [2]:
# load basic dependencies
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math, sys

In [3]:
# load data
train = pd.read_csv('/kaggle/input/pm25prediction/train.csv', encoding='gbk')
test = pd.read_csv('/kaggle/input/pm25prediction/test.csv', encoding='gbk', header=None)
predict = pd.read_csv('/kaggle/input/pm25prediction/predict.csv', encoding='gbk')
train.shape, test.shape, predict.shape

((4320, 27), (4320, 11), (240, 2))

In [4]:
test.head()

,0,1,2,3,4,5,6,7,8,9,10
0,id_0,AMB_TEMP,21,21,20,20,19,19,19,18,17
1,id_0,CH4,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.8
2,id_0,CO,0.39,0.36,0.36,0.4,0.53,0.55,0.34,0.31,0.23
3,id_0,NMHC,0.16,0.24,0.22,0.27,0.27,0.26,0.27,0.29,0.1
4,id_0,NO,1.3,1.3,1.3,1.3,1.4,1.6,1.2,1.1,0.9


In [5]:
def show_info(data, is_matrix_transpose=False):
    # basic shape
    print('data shape is: {}   sample number {}   attribute number {}\n'.format(data.shape, data.shape[0], data.shape[1]))
    # attribute(key)
    print('data columns number {}  \nall columns: {}\n'.format(len(data.columns) ,data.columns))
    # value's null
    print('data all attribute count null:\n', data.isna().sum())
    # data value analysis and data demo
    if is_matrix_transpose:
        print('data value analysis: ', data.describe().T)
        print('data demo without matrix transpose: ', data.head().T)
    else:
        print('data value analysis: ', data.describe())
        print('data demo without matrix transpose: ', data.head())
show_info(train)
show_info(test)
show_info(predict)

data shape is: (4320, 27)   sample number 4320   attribute number 27

data columns number 27  
all columns: Index(['日期', '測站', '測項', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21',
       '22', '23'],
      dtype='object')

data all attribute count null:
 日期    0
測站    0
測項    0
0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
dtype: int64
data value analysis:                 日期    測站    測項     0     1     2     3     4     5     6  ...  \
count        4320  4320  4320  4320  4320  4320  4320  4320  4320  4320  ...   
unique        240     1    18   369   361   351   355   353   342   356  ...   
top     2014/7/20    豐原    O3    NR    NR    NR    NR    NR    NR    NR  ...   
freq           18  4320   240   221   225   229   226   229   230   226  ...   

          1

## Data Preprocessing

* replace 'NR' with 0
* extract valid columns

* transform to numpy type
* transform data feature
    * 按照月份提取，先将月份拼接
    * 每九个小时作为一个sample，然后向后平移一个单位，与前一个Sample重叠八个小时做新的sample。=》10个小时可以提取1个sample，11个小时可以提取2个sample，一个月20天 * 24h = 480h可以提取471个sample，一共 12个月，共可以提取12 * 471 = 5652个sample;; attribution 共有18个 * 9h属性

* normalization

In [6]:
# replace 'NR' with 0
train[train == 'NR'] = 0
# extract valid columns
train = train.iloc[:, 3:]
train.head(30)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,14,14,14,13,12,12,12,12,15,17,...,22,22,21,19,17,16,15,15,15,15
1,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,0.51,0.41,0.39,0.37,0.35,0.3,0.37,0.47,0.78,0.74,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,0.2,0.15,0.13,0.12,0.11,0.06,0.1,0.13,0.26,0.23,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,0.9,0.6,0.5,1.7,1.8,1.5,1.9,2.2,6.6,7.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5
5,16,9.2,8.2,6.9,6.8,3.8,6.9,7.8,15,21,...,11,11,22,28,19,12,8.1,7,6.9,6
6,17,9.8,8.7,8.6,8.5,5.3,8.8,9.9,22,29,...,14,13,25,30,21,13,9.7,8.6,8.7,7.5
7,16,30,27,23,24,28,24,22,21,29,...,65,64,51,34,33,34,37,38,38,36
8,56,50,48,35,25,12,4,2,11,38,...,52,51,66,85,85,63,46,36,42,42
9,26,39,36,35,31,28,25,20,19,30,...,36,45,42,49,45,44,41,30,24,13


In [7]:
np_train = train.to_numpy()

In [8]:
month_train = {}
# 12 month
for month in range(12):
    tem = np.empty([18, 480])
    # every month has 20 days
    for day in range(20):
        tem[:, day*24: (day+1)*24] = np_train[18*(20*month+day): 18*(20*month+day+1), :]
    month_train[month] = tem

In [9]:
# declare X_train[12*471][18*9]
X_train = np.empty([12*471, 18*9], dtype=float)
y_train = np.empty([12*471, 1], dtype=float)
# extract feature from train data
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                continue
            X_train[month*471 + day*24 + hour, :] = month_train[month][:, day*24 + hour: day*24 + hour + 9].reshape(1, -1)
            y_train[month*471 + day*24 + hour, 0] = month_train[month][9, day*24 + hour + 9]
np.size(X_train, 0), np.size(X_train, 1), np.size(y_train, 0), np.size(y_train, 1)

(5652, 162, 5652, 1)

In [10]:
# normalization => X' = (X - mean)/std
mean_X_train = np.mean(X_train, axis=0)
std_X_train = np.std(X_train, axis=0)
for i in range(len(X_train)):
    for j in range(len(X_train[0])):
        if std_X_train[j] != 0:
            X_train[i][j] = (X_train[i][j] - mean_X_train[j]) / std_X_train[j]
X_train

array([[-1.35825331, -1.35883937, -1.359222  , ...,  0.26650729,
         0.2656797 , -1.14082131],
       [-1.35825331, -1.35883937, -1.51819928, ...,  0.26650729,
        -1.13963133, -1.32832904],
       [-1.35825331, -1.51789368, -1.67717656, ..., -1.13923451,
        -1.32700613, -0.85955971],
       ...,
       [-0.88092053, -0.72262212, -0.56433559, ..., -0.57693779,
        -0.29644471, -0.39079039],
       [-0.7218096 , -0.56356781, -0.72331287, ..., -0.29578943,
        -0.39013211, -0.1095288 ],
       [-0.56269867, -0.72262212, -0.88229015, ..., -0.38950555,
        -0.10906991,  0.07797893]])

In [11]:
# data preprocessing -- test

# extract valid columns
test = test.iloc[:, 2:]
# replace 'NR' with 0
test[test == 'NR'] = 0
# transform to numpy
test = test.to_numpy()

# declare data
X_test = np.empty([240, 18*9], dtype=float)
# flatten 18 row to 1 row
for i in range(240):
    X_test[i, :] = test[18*i: 18*(i+1), :].reshape(1, -1)
# normalization
mean_X_test = np.mean(X_test, axis=0)
std_X_test = np.std(X_test, axis=0)
for i in range(len(X_test)):
    for j in range(len(X_test[0])):
        if std_X_test[j] != 0:
            X_test[i][j] = (X_test[i][j] - mean_X_test[j]) / std_X_test[j]
X_test = np.concatenate((np.ones([240, 1]), X_test), axis=1).astype(float)
np.size(X_test, 0), np.size(X_test, 1)

(240, 163)

## build model

* train test split (train:val = 0.8)
* build model 
* gradient descent -- adagrad
* predict, submit
    * load weight
    * predict data

In [12]:
# train test split
X_train_set = X_train[: math.floor(len(X_train)*0.8), :]
y_train_set = y_train[: math.floor(len(y_train)*0.8), :]
X_val = X_train[math.floor(len(X_train)*0.8):, :]
y_val = y_train[math.floor(len(y_train)*0.8):, :]
np.size(X_train_set, 0), np.size(X_train_set, 1),np.size(y_train_set, 0), np.size(y_train_set, 1), np.size(X_val, 0), np.size(X_val, 1),np.size(y_val, 0), np.size(y_val, 1),

(4521, 162, 4521, 1, 1131, 162, 1131, 1)

In [13]:
dim = 18*9+1
weight = np.zeros([dim, 1])
X_train = np.concatenate((np.ones([12*471, 1]), X_train), axis=1).astype(float)
learning_rate = 100
iterations = 1000
adagrad = np.zeros([dim, 1])
eps = 0.0000000001
for t in range(iterations):
    loss = np.sqrt(np.sum(np.power(np.dot(X_train, weight) - y_train, 2))/471/12)
    gradient = 2*np.dot(X_train.transpose(), np.dot(X_train, weight) - y_train)
    adagrad += gradient ** 2
    weight = weight - learning_rate * gradient / np.sqrt(adagrad + eps)
np.save('weight.npy', weight)

In [14]:
# load weight
weight = np.load('weight.npy')
# predict
pred = np.dot(X_test, weight)
# print
pred

array([[ 4.6781933 ],
       [16.62707522],
       [12.57345236],
       [11.76672645],
       [19.71305601],
       [16.04124505],
       [15.98713649],
       [24.6198061 ],
       [ 7.68841459],
       [52.83128854],
       [17.44315819],
       [12.55894447],
       [53.6388442 ],
       [34.78785901],
       [12.14537545],
       [ 6.83714017],
       [22.80579212],
       [55.10165077],
       [-6.88475471],
       [13.70131281],
       [29.37878281],
       [56.30604445],
       [ 4.56477556],
       [13.29520107],
       [11.79705528],
       [27.07474796],
       [14.19006585],
       [59.01327628],
       [ 9.54694657],
       [44.76286607],
       [19.55470217],
       [ 0.89452631],
       [ 4.62957481],
       [20.51740181],
       [22.25733321],
       [29.22043288],
       [35.27425107],
       [22.93788281],
       [31.9821482 ],
       [19.23803482],
       [ 2.32064936],
       [30.13214112],
       [28.00107828],
       [37.94624292],
       [10.51598291],
       [25